Get all Bathing Waters with P9618 in this project from Wikidata and checks if they exist on eionet as a WaterBody 

https://dd.eionet.europa.eu/vocabularyconcept/wise/WaterBody

* The project: [github salgo60/Svenskabadplatser](https://github.com/salgo60/Svenskabadplatser)
  * European bathwaters [GITHUB](https://github.com/salgo60/EuropeanBathingWater/blob/main/README.md) / [Wikidata](https://www.wikidata.org/wiki/Wikidata:WikiProject_European_Bath_Waters)
* this [Notebook](https://github.com/salgo60/Svenskabadplatser/blob/main/Jupyter/Eionet%20Data%20Dictionary.ipynb)

  
Status:  



| Date | Total | Ok | Error 
| ------------- |:-------------:|:-------------:|:-------------:|
| 20210610 | 3176 | 936 | 2240 |


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-06-12 08:49:54.218008


In [2]:
import pandas as pd


In [3]:
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/3Tk$    
queryBath = """SELECT  (REPLACE(STR(?nodebath), ".*Q", "Q") AS ?wikidata) ?nodebath (SUBSTR(lcase(?bath),1,2) AS ?country)
(URI(CONCAT("https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.",
       str(?bath))) AS ?eionet)  (str(?bath) AS ?bathwateridentifier){
      ?nodebath wdt:P9616 ?bath} 
"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDBath = get_sparql_dataframe(endpoint_url, queryBath)
WDBath.shape

(3175, 5)

In [4]:
WDBath.head()

wikidata                                   nodebath country  \
0  Q106708785  http://www.wikidata.org/entity/Q106708785      se   
1  Q106708786  http://www.wikidata.org/entity/Q106708786      se   
2  Q106708787  http://www.wikidata.org/entity/Q106708787      se   
3  Q106708789  http://www.wikidata.org/entity/Q106708789      se   
4  Q106708783  http://www.wikidata.org/entity/Q106708783      se   

                                              eionet bathwateridentifier  
0  https://dd.eionet.europa.eu/vocabularyconcept/...  SE0230512000001917  
1  https://dd.eionet.europa.eu/vocabularyconcept/...  SE0230512000001918  
2  https://dd.eionet.europa.eu/vocabularyconcept/...  SE0230512000001919  
3  https://dd.eionet.europa.eu/vocabularyconcept/...  SE0A21493000001925  
4  https://dd.eionet.europa.eu/vocabularyconcept/...  SE0230512000001915

In [5]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()
urlHav = "https://badplatsen.havochvatten.se/badplatsen/api/detail/" 

listBath = []
for WD, row in tqdm(WDBath.iterrows(), total=WDBath.shape[0]):
    url = row["eionet"] 
    
    new_item = dict()
    new_item['wikidata'] = row["wikidata"] 
    #print(url)
    try:
        r = http.request('GET', url) 
        new_item['status'] = r.status
        if  r.status == 404 and row['country'] == "se":
            #Check if Sweden. and check API for reason
            try:
                urlHavBath = urlHav + row["bathwateridentifier"]
                rHav = http.request('GET',urlHavBath , 
                                    headers={'Content-Type': 'application/json'})
                rHavData = json.loads(rHav.data.decode('utf-8'))  
                #for key, value in rHavData.items() :
                #    print ("\t\t",key, value)
                new_item['euType'] = rHavData["euType"]
                new_item['euMotive'] = rHavData["euMotive"]
                new_item['NotEuMotive'] = rHavData["NotEuMotive"]
                
            except Exception as e:
                print ("Hav except ", e, urlHavBath, " WD:",row["wikidata"] )

    except:
        #print (r.status, url)
        new_item['status'] = r.status
    new_item['eionet'] = url 
    new_item['bathwateridentifier'] = row["bathwateridentifier"] 
    new_item['country'] = row["country"] 
    
    listBath.append(new_item)
print (len(listBath))

 38%|███▊      | 1191/3175 [01:08<02:23, 13.79it/s]

Hav except  Expecting value: line 1 column 1 (char 0) https://badplatsen.havochvatten.se/badplatsen/api/detail/SE0920780000001554  WD: Q106708515


 38%|███▊      | 1197/3175 [01:09<02:29, 13.23it/s]

Hav except  Expecting value: line 1 column 1 (char 0) https://badplatsen.havochvatten.se/badplatsen/api/detail/SE0920780000001564  WD: Q106708524


 43%|████▎     | 1373/3175 [01:16<01:58, 15.21it/s]

Hav except  Expecting value: line 1 column 1 (char 0) https://badplatsen.havochvatten.se/badplatsen/api/detail/SE0110126000005596  WD: Q106673424


 62%|██████▏   | 1957/3175 [01:50<01:27, 13.84it/s]

Hav except  Expecting value: line 1 column 1 (char 0) https://badplatsen.havochvatten.se/badplatsen/api/detail/SE0A21439000001358  WD: Q106708374
Hav except  Expecting value: line 1 column 1 (char 0) https://badplatsen.havochvatten.se/badplatsen/api/detail/SE0A21439000001360  WD: Q106708376


100%|██████████| 3175/3175 [02:59<00:00, 17.66it/s]

3175


In [6]:
#listBath

In [7]:
Eionettot = pd.DataFrame(listBath,
                  columns=['wikidata','country','bathwateridentifier','status','eionet','euType','euMotive','NotEuMotive'])
Eionettot.shape


(3175, 8)

In [8]:
Eionettot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3175 entries, 0 to 3174
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   wikidata             3175 non-null   object 
 1   country              3175 non-null   object 
 2   bathwateridentifier  3175 non-null   object 
 3   status               3175 non-null   int64  
 4   eionet               3175 non-null   object 
 5   euType               2202 non-null   object 
 6   euMotive             17 non-null     object 
 7   NotEuMotive          0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 198.6+ KB


In [9]:
pd.set_option('max_colwidth', 400)
Eionettot.head(10)

wikidata country bathwateridentifier  status  \
0  Q106708785      se  SE0230512000001917     404   
1  Q106708786      se  SE0230512000001918     404   
2  Q106708787      se  SE0230512000001919     404   
3  Q106708789      se  SE0A21493000001925     404   
4  Q106708783      se  SE0230512000001915     404   
5  Q106708788      se  SE0230512000001920     404   
6  Q106708790      se  SE0930821000001933     404   
7  Q106708784      se  SE0230512000001916     404   
8  Q106708793      se  SE0930821000001936     404   
9  Q106708791      se  SE0930821000001934     404   

                                                                                                       eionet  \
0  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0230512000001917   
1  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0230512000001918   
2  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0230512000001919   
3  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0A21493000001925   
4  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0230512000001915   
5  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0230512000001920   
6  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0930821000001933   
7  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0230512000001916   
8  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0930821000001936   
9  https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode.SE0930821000001934   

  euType euMotive  NotEuMotive  
0  False     None          NaN  
1  False     None          NaN  
2  False     None          NaN  
3  False     None          NaN  
4  False     None          NaN  
5  False     None          NaN  
6  False     None          NaN  
7  False     None          NaN  
8  False     None          NaN  
9  False     None          NaN

In [10]:
#Eionettot["link"] = "<a href='https://dd.eionet.europa.eu/vocabularyconcept/wise/WFDProtectedArea/euProtectedAreaCode." + Eionettot["eionet"].astype(str) + "'">link eionet</a>"
Eionettot["link"] = "<a href='" + Eionettot["eionet"].astype(str) + "'>link eionet</a>"
Eionettot["WD"] = "<a href='https://www.wikidata.org/wiki/" + Eionettot["wikidata"].astype(str) + "'>link WD</a>"


In [11]:
from IPython.display import display, HTML  

#Eionettot.value_counts({"status","country"})
#Eionettot[['status', 'country']].apply(pd.Series.value_counts)
HTML(Eionettot[{'WD','country','bathwateridentifier','status','link','euType','euMotive','NotEuMotive'}].tail(50).to_html(escape=False))

In [12]:
EionettotOk = Eionettot[(Eionettot['status']==200)] 
EionettotError = Eionettot[(Eionettot['status']==404)]

In [13]:
EionettotOk.shape

(936, 10)

In [14]:
EionettotError.shape

(2239, 10)

In [15]:
EionettotError.value_counts("country")

country
se    2207
es      13
gr      12
hr       4
de       1
ie       1
uk       1
dtype: int64

In [16]:
EionettotError.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2239 entries, 0 to 3173
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   wikidata             2239 non-null   object 
 1   country              2239 non-null   object 
 2   bathwateridentifier  2239 non-null   object 
 3   status               2239 non-null   int64  
 4   eionet               2239 non-null   object 
 5   euType               2202 non-null   object 
 6   euMotive             17 non-null     object 
 7   NotEuMotive          0 non-null      float64
 8   link                 2239 non-null   object 
 9   WD                   2239 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 192.4+ KB


In [17]:

HTML(EionettotError[{'WD','country','bathwateridentifier','status','link','euType','euMotive','NotEuMotive'}].tail(10).to_html(escape=False))

In [18]:
EionettotErrorEuType = EionettotError[EionettotError["euType"] == True] 
HTML(EionettotErrorEuType[{'WD','country','bathwateridentifier','status','link','euType','euMotive','NotEuMotive'}].tail(10).to_html(escape=False))

In [19]:
EionettotError["euMotive"].value_counts()

<200 badande per dag. Ej längre EU bad.                                                                                                                                                                                                                                                                                                                                                                            3
Stora investeringar 2019 med en väl tilltagen brygga i ett populärt område i norra delarna av Kalmar. Fler än 200 badande per dag i snitt 2019.                                                                                                                                                                                                                                                                    1
Antalet besökare uppskattas till runt 500 personer fina sommardagar                                                                                                                           

In [20]:
EionettotOk.shape

(936, 10)

In [21]:
EionettotOk.value_counts("country")

country
se    487
dk     36
de     35
pt     33
fr     27
ie     24
it     23
lt     22
fi     21
uk     21
es     21
nl     20
pl     20
hu     17
ee     17
si     16
bg     14
ro     11
hr     10
at      9
lv      9
sk      8
al      8
lu      8
be      7
cz      6
mt      6
dtype: int64

In [22]:
EionettotOk.to_csv("BathIdentifier_Ok.csv")
EionettotError.to_csv("BathIdentifier_Error.csv")
Eionettot.to_csv("BathIdentifier_All.csv")


Generate Markdown table eg.
| 20210610 | 3176 | 2240 | 936 |


In [23]:
print("|",start_time.strftime("%Y%m%d"),"|", \
      Eionettot.shape[0],"|", \
      EionettotOk.shape[0],"|", \
      EionettotError.shape[0],"|",)


| 20210612 | 3175 | 936 | 2239 |


In [24]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-06-12 08:52:56.770512
Time elapsed (hh:mm:ss.ms) 0:03:02.552656
